# Running VMAP Stability Test on Sherbrooke

In [1]:
# Importing Packages

import numpy as np
import matplotlib.pyplot as plt
import time

from typing import Optional, Union

import jax
import jax.numpy as jnp
from jax.scipy.special import erf
from jax import jit, vmap, block_until_ready

from utils import plot_learning, photon_env_dicts

In [2]:
# Seeding everything

seed = 30

rng = jax.random.PRNGKey(seed)
rng, _rng = jax.random.split(rng)

In [3]:
import jax
import jax.numpy as jnp
from jax import jit, config, vmap, block_until_ready
import flax.linen as nn
import numpy as np
import optax
from flax.linen.initializers import constant, orthogonal
from typing import Sequence, NamedTuple, Any
from flax.training.train_state import TrainState
import distrax
import chex

from rl_algos.rl_wrappers import VecEnv

from envs.photon_env import BatchedPhotonLangevinReadoutEnv
from envs.single_photon_env import SinglePhotonLangevinReadoutEnv

import time

import matplotlib.pyplot as plt

from utils import photon_env_dicts


class SeparateActorCritic(nn.Module):
    """
    Actor and Critic with Separate Feed-forward Neural Networks
    """

    action_dim: Sequence[int]
    activation: str = "tanh"
    layer_size: int = 128

    @nn.compact
    def __call__(self, x):
        if self.activation == "relu":
            activation = nn.relu
        if self.activation == "elu":
            activation = nn.elu
        if self.activation == "leaky_relu":
            activation = nn.leaky_relu
        if self.activation == "relu6":
            activation = nn.relu6
        if self.activation == "selu":
            activation = nn.selu
        else:
            activation = nn.tanh
        actor_mean = nn.Dense(
            self.layer_size, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(x)
        actor_mean = activation(actor_mean)
        actor_mean = nn.Dense(
            self.layer_size, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(actor_mean)
        actor_mean = activation(actor_mean)
        actor_mean = nn.Dense(
            self.action_dim, kernel_init=orthogonal(0.01), bias_init=constant(0.0)
        )(actor_mean)
        actor_logtstd = self.param("log_std", nn.initializers.zeros, (self.action_dim,))
        pi = distrax.MultivariateNormalDiag(actor_mean, jnp.exp(actor_logtstd))

        critic = nn.Dense(
            self.layer_size, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(x)
        critic = activation(critic)
        critic = nn.Dense(
            self.layer_size, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(critic)
        critic = activation(critic)
        critic = nn.Dense(1, kernel_init=orthogonal(1.0), bias_init=constant(0.0))(
            critic
        )

        return pi, jnp.squeeze(critic, axis=-1)


class CombinedActorCritic(nn.Module):
    """
    Actor and Critic Class with combined Feed-forward Neural Network
    """

    action_dim: Sequence[int]
    activation: str = "tanh"
    layer_size: int = 128

    @nn.compact
    def __call__(self, x):
        if self.activation == "relu":
            activation = nn.relu
        if self.activation == "elu":
            activation = nn.elu
        if self.activation == "leaky_relu":
            activation = nn.leaky_relu
        if self.activation == "relu6":
            activation = nn.relu6
        if self.activation == "selu":
            activation = nn.selu
        else:
            activation = nn.tanh
        actor_mean = nn.Dense(
            self.layer_size, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(x)
        actor_mean = activation(actor_mean)
        actor_mean = nn.Dense(
            self.layer_size, kernel_init=orthogonal(np.sqrt(2)), bias_init=constant(0.0)
        )(actor_mean)
        actor_mean = activation(actor_mean)
        actor_mean_val = nn.Dense(
            self.action_dim, kernel_init=orthogonal(0.01), bias_init=constant(0.0)
        )(actor_mean)
        actor_logtstd = self.param("log_std", nn.initializers.zeros, (self.action_dim,))
        pi = distrax.MultivariateNormalDiag(actor_mean_val, jnp.exp(actor_logtstd))

        critic = nn.Dense(1, kernel_init=orthogonal(1.0), bias_init=constant(0.0))(
            actor_mean
        )

        return pi, jnp.squeeze(critic, axis=-1)


class Transition(NamedTuple):
    """
    Class for carrying RL State between processes
    """

    # done: jnp.ndarray
    action: jnp.ndarray
    value: jnp.ndarray
    reward: jnp.ndarray
    log_prob: jnp.ndarray
    obs: jnp.ndarray
    # info: jnp.ndarray


def kc_PPO_make_train(config):
    """
    Function that returns a trainable function for an input configuration dictionary
    """
    env_dict = photon_env_dicts()

    config["MINIBATCH_SIZE"] = (
        config["NUM_ENVS"] * config["NUM_STEPS"] // config["NUM_MINIBATCHES"]
    )

    def linear_schedule(count):
        frac = (
            1.0
            - (count // (config["NUM_MINIBATCHES"] * config["UPDATE_EPOCHS"]))
            / config["NUM_UPDATES"]
        )
        return config["LR"] * frac

    env_params = None

    if config["ANNEAL_LR"]:
        tx = optax.chain(
            optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
            optax.adam(learning_rate=linear_schedule, eps=1e-5),
        )
    else:
        tx = optax.chain(
            optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
            optax.adam(config["LR"], eps=1e-5),
        )

    def kc_train(
        rng: chex.PRNGKey,
        kappa: float,
        chi: float,
        num_envs: int,
    ):
        """
        Training function for environment
        """
        env = env_dict[config["ENV_NAME"]](
            kappa=kappa, 
            chi=chi,
            **config["ENV_PARAMS"]
        )
        env = VecEnv(env)
        # INIT NETWORK
        network = CombinedActorCritic(
            env.action_space(env_params).shape[0],
            activation=config["ACTIVATION"],
            layer_size=config["LAYER_SIZE"],
        )
        rng, _rng = jax.random.split(rng)
        init_x = jnp.zeros(env.observation_space(env_params).shape)
        network_params = network.init(_rng, init_x)
        train_state = TrainState.create(
            apply_fn=network.apply,
            params=network_params,
            tx=tx,
        )

        # INIT ENV
        rng, _rng = jax.random.split(rng)
        reset_rng = jax.random.split(_rng, num_envs)
        obsv, env_state = env.reset(reset_rng, env_params)

        # We only use init_x as the batched observation, and env_state for state information

        # TRAIN LOOP
        def _update_step(runner_state, unused):
            # CALCULATE TRAJECTORIES
            train_state, env_state, batched_last_obs, rng = runner_state

            # SELECT BATCHED ACTIONS
            rng, _rng = jax.random.split(rng)
            pi, batched_value = network.apply(train_state.params, batched_last_obs)
            batched_action = pi.sample(seed=_rng)
            batched_log_prob = pi.log_prob(batched_action)

            rng, _rng = jax.random.split(rng)
            rng_step = jax.random.split(_rng, num_envs)
            returned_obsv, env_state, batched_reward, done, batch_info = env.step(
                rng_step, env_state, batched_action, env_params
            )
            batched_transition = Transition(
                batched_action,
                batched_value,
                batched_reward,
                batched_log_prob,
                batched_last_obs,
            )

            # We pass the old observations here, it doesn't change in the single-step environment
            runner_state = (train_state, env_state, batched_last_obs, rng)

            batched_advantage = batched_reward - batched_value

            # UPDATE NETWORK
            def _update_epoch(update_state, unused):
                def _update_minbatch(train_state, batch_info):
                    traj_batch, advantages, targets = batch_info

                    def _loss_fn(params, traj_batch, gae, targets):
                        # RERUN NETWORK
                        pi, value = network.apply(params, traj_batch.obs)
                        log_prob = pi.log_prob(traj_batch.action)

                        # CALCULATE VALUE LOSS
                        value_pred_clipped = traj_batch.value + (
                            value - traj_batch.value
                        ).clip(-config["VALUE_CLIP_EPS"], config["VALUE_CLIP_EPS"])
                        value_losses = jnp.square(value - targets)
                        value_losses_clipped = jnp.square(value_pred_clipped - targets)
                        value_loss = (
                            0.5 * jnp.maximum(value_losses, value_losses_clipped).mean()
                        )

                        # CALCULATE ACTOR LOSS
                        ratio = jnp.exp(log_prob - traj_batch.log_prob)
                        gae = (gae - gae.mean()) / (gae.std() + 1e-8)
                        loss_actor1 = ratio * gae
                        loss_actor2 = (
                            jnp.clip(
                                ratio,
                                1.0 - config["CLIP_EPS"],
                                1.0 + config["CLIP_EPS"],
                            )
                            * gae
                        )
                        loss_actor = -jnp.minimum(loss_actor1, loss_actor2)
                        loss_actor = loss_actor.mean()
                        entropy = pi.entropy().mean()

                        total_loss = (
                            loss_actor
                            + config["VF_COEF"] * value_loss
                            - config["ENT_COEF"] * entropy
                        )
                        return total_loss, (value_loss, loss_actor, entropy)

                    grad_fn = jax.value_and_grad(_loss_fn, has_aux=True)
                    total_loss, grads = grad_fn(
                        train_state.params, traj_batch, advantages, targets
                    )
                    train_state = train_state.apply_gradients(grads=grads)
                    return train_state, total_loss

                train_state, traj_batch, advantages, targets, rng = update_state
                rng, _rng = jax.random.split(rng)
                batch_size = num_envs
                assert (
                    batch_size == config["NUM_STEPS"] * config["NUM_ENVS"]
                ), "batch size must be equal to number of steps * number of envs"
                permutation = jax.random.permutation(_rng, batch_size)
                batch = (traj_batch, advantages, targets)
                batch = jax.tree_util.tree_map(
                    lambda x: x.reshape((batch_size,) + x.shape[1:]), batch
                )
                shuffled_batch = jax.tree_util.tree_map(
                    lambda x: jnp.take(x, permutation, axis=0), batch
                )
                minibatches = jax.tree_util.tree_map(
                    lambda x: jnp.reshape(
                        x, [config["NUM_MINIBATCHES"], -1] + list(x.shape[1:])
                    ),
                    shuffled_batch,
                )
                train_state, total_loss = jax.lax.scan(
                    _update_minbatch, train_state, minibatches
                )
                update_state = (train_state, traj_batch, advantages, targets, rng)
                return update_state, total_loss

            update_state = (
                train_state,
                batched_transition,
                batched_advantage,
                batched_reward,
                rng,
            )
            update_state, loss_info = jax.lax.scan(
                _update_epoch, update_state, None, config["UPDATE_EPOCHS"]
            )
            train_state = update_state[0]
            metric = batch_info
            global_updatestep = metric["timestep"][0]
            rng = update_state[-1]
            if config.get("DEBUG"):

                def return_readout_stats(global_updatestep, info):
                    jax.debug.print("global update: {update}", update=global_updatestep)
                    jax.debug.print(
                        "reward: {reward}",
                        reward=jnp.round(jnp.mean(info["reward"]), 3),
                    )
                    jax.debug.print(
                        "max pF: {pF}", pF=jnp.round(jnp.mean(info["max pF"]), 3)
                    )
                    jax.debug.print(
                        "max photon: {photon}",
                        photon=jnp.round(jnp.mean(info["max photon"]), 3),
                    )
                    jax.debug.print(
                        "photon time: {time}",
                        time=jnp.round(jnp.mean(info["photon time"]), 4),
                    )
                    jax.debug.print(
                        "smoothness: {smoothness}",
                        smoothness=jnp.round(jnp.mean(info["smoothness"]), 6),
                    )
                    jax.debug.print(
                        "bandwidth: {bandwidth}",
                        bandwidth=jnp.round(jnp.mean(info["bandwidth"]), 3),
                    )
                    jax.debug.print(
                        "pulse reset val: {p}",
                        p=jnp.round(jnp.mean(info["pulse reset val"]), 3),
                    )

                def pass_stats(global_updatestep, info):
                    pass

                def return_action(global_updatestep, info):
                    jax.debug.print(
                        "action of max={action_of_max}",
                        action_of_max=info["action of max"],
                    )

                jax.lax.cond(
                    global_updatestep % config["PRINT_RATE"] == 0,
                    return_readout_stats,
                    pass_stats,
                    global_updatestep,
                    metric,
                )
                if config.get("DEBUG_ACTION"):
                    jax.lax.cond(
                        global_updatestep % config["ACTION_PRINT_RATE"] == 0,
                        return_action,
                        pass_stats,
                        global_updatestep,
                        metric,
                    )

            runner_state = (train_state, env_state, batched_last_obs, rng)
            return runner_state, metric

        rng, _rng = jax.random.split(rng)
        runner_state = (train_state, env_state, obsv, _rng)
        runner_state, metric = jax.lax.scan(
            _update_step, runner_state, None, config["NUM_UPDATES"]
        )
        return {"runner_state": runner_state, "metrics": metric}

    return kc_train

In [4]:
# Defining Sherbrooke Params and RL Params

tau_0 = 0.783
kappa = 14.31
chi = 0.31 * 2. * jnp.pi
kerr = 0.00
n0 = 53.8
res_amp_scaling = 1/0.348
actual_max_photons = n0 * (1. - 2. * jnp.cos(0.5 * chi * tau_0) * jnp.exp(-0.5 * kappa * tau_0) + jnp.exp(-0.5 * kappa * tau_0))
print(f"Rough Max Photons: {n0}")
print(f"Actual Max Photons: {actual_max_photons}")
nR = 0.05
snr_scale_factor = 1.25
gamma_I = 1/362.9
photon_gamma = 1/4000
init_fid = 1.

time_coeff = 10.0
snr_coeff = 20.0
smoothness_coeff = 10.0
smoothness_baseline_scale = 1.0
gauss_kernel_len = 15
gauss_kernel_std = 2.0
bandwidth = 50.0
freq_relative_cutoff = 0.1
bandwidth_coeff = 0.0
num_t1 = 9.0
photon_weight = 12.0
shot_noise_std = 0.0
standard_fid = 0.99

reduced_env_config = {
    "kerr": kerr,
    "time_coeff": time_coeff,
    "snr_coeff": snr_coeff,
    "smoothness_coeff": smoothness_coeff,
    "smoothness_baseline_scale": smoothness_baseline_scale,
    "gauss_kernel_len": gauss_kernel_len,
    "gauss_kernel_std": gauss_kernel_std,
    "bandwidth": bandwidth,
    "freq_relative_cutoff": freq_relative_cutoff,
    "bandwidth_coeff": bandwidth_coeff,
    "n0": n0,
    "tau_0": tau_0,
    "res_amp_scaling": res_amp_scaling,
    "nR": nR,
    "snr_scale_factor": snr_scale_factor,
    "gamma_I": gamma_I,
    "photon_gamma": photon_gamma,
    "num_t1": num_t1,
    "init_fid": init_fid,
    "photon_weight": photon_weight,
    "standard_fid": standard_fid,
    "shot_noise_std": shot_noise_std,
}

num_envs = 256
num_updates = 4000
config = {
    "LR": 3e-4,
    "NUM_ENVS": num_envs,
    "NUM_STEPS": 1,
    "NUM_UPDATES": num_updates,
    "UPDATE_EPOCHS": 4,
    "NUM_MINIBATCHES": int(num_envs / 64),
    "CLIP_EPS": 0.2,
    "VALUE_CLIP_EPS": 0.2,
    "ENT_COEF": 0.0,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "ACTIVATION": "relu6",
    "LAYER_SIZE": 64,
    "ENV_NAME": "single_langevin_env",
    "ENV_PARAMS": reduced_env_config,
    "ANNEAL_LR": False,
    "DEBUG": True,
    "DEBUG_ACTION": False,
    "PRINT_RATE": 10,
    "ACTION_PRINT_RATE": 100,
}

Rough Max Photons: 53.8
Actual Max Photons: 53.711451977836106


In [5]:
e = 0.05
kappas = kappa * jnp.array([1. - e, 1., 1. + e])
chis = chi * jnp.array([1. - e, 1., 1. + e])
kc_grid = jnp.array(jnp.meshgrid(kappas, chis)).reshape(2, -1).T
kappa_run = kc_grid[:, 0]
chi_run = kc_grid[:, 1]

print(kappa_run)
print(chi_run)

[13.5945 14.31   15.0255 13.5945 14.31   15.0255 13.5945 14.31   15.0255]
[1.85039807 1.85039807 1.85039807 1.94778745 1.94778745 1.94778745
 2.04517682 2.04517682 2.04517682]


In [6]:
vmap_train = vmap(kc_PPO_make_train(config), in_axes=(0, 0, 0, None))
jit_vmap_train = jit(vmap_train, static_argnums=-1)

rng, _rng = jax.random.split(rng)
rng_vmap = jax.random.split(_rng, len(kappa_run))

print(f"Starting a Run of {num_updates} Updates")
start = time.time()
stability_result = jit_vmap_train(
    rng_vmap,
    kappa_run,
    chi_run,
    num_envs)
end = time.time()
print(f"time taken: {end - start}")

Starting a Run of 4000 Updates
global update: 10
reward: -162.76
reward: -159.91400000000002
reward: -161.839
reward: -163.75
reward: -162.667
reward: -149.96
reward: -174.63
reward: -158.627
reward: -160.489
max pF: 0.493
max pF: 0.488
max pF: 0.467
max pF: 0.498
max pF: 0.485
max pF: 0.47900000000000004
max pF: 0.511
max pF: 0.504
max pF: 0.488
max photon: 3.9610000000000003
max photon: 4.095
max photon: 3.6990000000000003
max photon: 3.777
max photon: 3.666
max photon: 3.9370000000000003
max photon: 3.829
max photon: 3.86
max photon: 3.729
photon time: 0.2892
photon time: 0.3003
photon time: 0.2995
photon time: 0.2807
photon time: 0.2785
photon time: 0.2967
photon time: 0.269
photon time: 0.2651
photon time: 0.2524
smoothness: 0.008659
smoothness: 0.007755
smoothness: 0.006764999999999999
smoothness: 0.008752
smoothness: 0.007607
smoothness: 0.006985
smoothness: 0.009047
smoothness: 0.0077269999999999995
smoothness: 0.006926
bandwidth: 48.264
bandwidth: 48.15
bandwidth: 49.095
bandw